# Sympy sympy sympy

We will continue to explore Sympy's capabilities, including:

* Calculus
* ODEs
* Converting Sympy expressions in to callable functions, to let us do things like plotting.
* Solving systems of equations. . . with and without sympy. 

* * *

### A quick aside: Jupyter notebook editing keyboard shortcuts

A complete list of keyboard shortcuts is obtained by clicking **Help -> Keyboard Shortcuts** above. 

* * *

## Back to calculus!

We left off with the *error function* also known as **erf**!

In [ ]:
import sympy as sp
x = sp.Symbol('x')

f = sp.exp(-x**2)
sp.pprint(f)

fi = sp.integrate(f,x)
sp.pprint(fi)

This is better than no information at all. sympy is telling us that (a rescaling) of this is called the $erf$ or <a href="http://docs.sympy.org/0.7.1/modules/mpmath/functions/expintegrals.html#erf">*error function*</a>.  Sympy can work with this function. 

Whenever you need to know what a sympy (or Python) object is simply type in **object?**, for example **sp.erf?**

We can similarly ask for definite integrals. 

We will request $\int_{-\infty}^\infty e^{-x^2} dx$ below.

Let's also request $\int_{-1}^1 e^{-x^2} dx$

i.e. when sympy fails on these kinds of requests, it **gives up** and returns your original input, *unchanged*. 

What is going on *under the hood* is that sympy is using something called the <a href="https://en.wikipedia.org/wiki/Risch_algorithm">**Risch algorithm**</a>. Technically sympy has developed an extension of the Risch algorithm. . . and sympy's extension what is known as a *semi-algorithm*.  As far as I am aware, every software package that can compute symbolic integrals (Mathematical, Matlab, Maple, etc) all use variants of the Risch algorithm, which similarly boils-down to a careful application of <a href="https://en.wikipedia.org/wiki/Liouville%27s_theorem_(differential_algebra)">Liouville's Theorem in differential algebra</a>. 

The **Liouville Theorem** states that if $f(x)$ has an anti-derivative that is *elementary* (a sum, product, quotient, or composite of functions involving exponential or logarithm, polynomials or trig functions *recursively*) then there is and expression for the function $F$ with $F'=f$ of the form:

$$ F(x) = v(x) + \sum_{i=1}^n c_i \ln(g_i(x)) $$

where $v(x)$ and the functions $g_1(x),\cdots,g_n(x)$ are elementary function expressible entirely in terms of sums, product, and quotients of $f(x)$, and polynomials.  

* * *

**eg:** $$\frac{e^{x^2+2}+2}{\ln(x)+x^8+\sin(e^x+x^2)} \hskip 1cm \text{ and } \hskip 1cm \cos\left(\frac{1}{1 + \tan\left(\frac{1}{\sin(x) + e^x}\right)}\right)$$
are elementary functions. 

**eg:** It is a fact that functions such as
$$\int e^{-x^2} dx \hskip 1cm \text{ and } \hskip 1cm \int \sin(x^2) dx $$
are **not** elementary.  

* * *

The Risch algorithm goes one step further and reduces the number of possibilities one has to consider to a finite number.  In Risch's original paper he confined himself to a fairly simple class of elementary functions on which his work provides a genuine algorithm. Sympy, on the other hand, has looser constraints than Risch's original paper and so it does not know how to solve this problem for every possible input. Sympy **can fail**, which is why it is called a semi-algorithm.

When you provide sympy with a *difficult* integral that it does not know how to handle immediately, it spends some time *searching* for an answer, but if it does not find one quickly it gives up. 

Sympy's extensions to the Risch algorithm are some of the most commonly-used anti-derivatives, such as $erf$ and the Fresnel function.

* * *

# Asking sympy to solve equations

Sympy has some fairly sophisticated algorithms to solve polynomial equations. It uses this intelligence for solving polynomial equations to build tools to solve (symbolically) a wide array of equations, even ones that are not polynomial. 

Sympy can:

 * Factor polynomials.
 * Find roots of polynomials, symbolically as well as numerically. 
 * Solve (symbolically as well as numerically) simultaneous polynomial equations.
 * Solve simultaneous equations that are not polynomial
     - sympy can *sometimes* do this symbolically
     - can usually do this numerically, using a variety of methods, including the multi-variable Newton's method.  The numeric methods are not always guanteed to find all solutions. If you *need* all solutions you might have to use more specialized methods. 
 * Sympy's polynomial equation solvers, on the other hand, will always find good approximations to all the solutions of a single-variable polynomial.

In [ ]:
p = x*x - 2
sp.pprint(sp.solve(p,x))

p = x*x + 2
sp.pprint(sp.solve(p,x))

### sympy solve

The general format is to call $solve(f,x)$ where $f$ is a sympy expression -- a function -- and $x$ is the variable used by that function.  Sympy will attempt to find all the solutions to the equation
$$f(x) = 0$$

The solve algorithm runs out of steam fairly quickly. For example:


## roots

The sympy *polyroots* and numpy *roots* functions are guanteed to find all the solutions of your single-variable polynomial equation.  

* * * 

While we're at it, notice mpmath can be used to solve multi-variable equations.

Here we ask mpmath to solve
$$(x^2+y^2, xy) = (4, 1)$$

It is using a multi-variable Newton method.

## Summary

Sympy has a systematic way of solving algebraic equations.  

 * At its core is a strong semi-algorithm to solve systems of polynomial equations.
     - An example system: $x^3+3xy+1=0 = y^4-x^2y+x+4 $
     - To solve such equations Sympy uses a **Groebner basis algorithm** to convert such systems of *multi-variable polynomial* equations into systems of *single-variable* polynomial equations. This is a reliable algorithm, although it can be slow (double-exponential run-time estimates).
     - Sympy does not know how to write "closed-form" solutions to all single variable polynomial equations, but it has a significant repository of closed-form solutions, as we have seen. 
     - As a side-note, it is *known* that the solutions to a general degree $5$ (or higher) polynomial equation $a_5x^5+a_4x^4+a_3x^3+a_2x^2+a_1x+a_0=0$ can not be expressed as a rational polynomial in rational powers (recursively) of $a_5,\cdots,a_0$, although as we have seen above this **can** be done if the degree is $4$ or less.   So one generally *can not* hope for simple formulas for such roots.

 * For more complicated systems, such as $\sin(x)^3+3\sin(x)\cos(y)+1=0=\sin(y)^4-\cos(x)^2+\cos(x)+4$ sympy thinks of the system as a polynomial equation in the variables $\sin(x), \cos(y)$ and computes $\sin^{-1}$ and $\cos^{-1}$ of the solutions.
 
 * Sympy also has adapted versions of the above algorithms to find closed-form solutions to some differential equations.  We will talk more about this when we discuss simulating solutions to differential equations.  One particularly simple case appears below.

* * *

# Sympy and ODEs

Sympy's algorithms for finding *anti-derivatives* have been adapted to algorithms for finding *closed-form* solutions to differential equations.  

These algorithms are strongest for **ODEs** but there are also algorithms for **PDEs**, delay differential equations, and types of differential equations. 

We start by exploring the [skydiver problem](../Week.4/Lecture.2.ipynb) again. 

$$ v' = kv^2 -g $$

## More Generally -- Solving ODES with sympy

At present, sympy's ode solving algorithm is basically a big *cookbook*-style database of formulas.  The key computational task sympy must perform is to recognise what type of differential equation you have provided it.  

Once it recognises the form from its cookbook, it follows standard procedures, which usually amount to either computing anti-derivatives, Fourier transforms, power series, etc. 

**Sympy has algorithms to solve:**

* First order ODEs that are: 
     - separable differential equations
     - differential equations whose coefficients homogeneous of the same order.
     - exact differential equations.
     - linear differential equations
     - Bernoulli differential equations.

* Second order ODEs that are:
    - Liouville differential equations.

* n-th order ODEs that are:
    - linear homogeneous differential equation with constant coefficients.
    - linear inhomogeneous differential equation with constant coefficients using the method of undetermined coefficients.
    - linear inhomogeneous differential equation with constant coefficients using the method of variation of parameters.

Sympy also has algorithms to solve some [PDEs](http://docs.sympy.org/latest/modules/solvers/pde.html), Delay Differential Equations [DDEs](http://users.ox.ac.uk/~clme1073/python/PyDDE/) and pretty much any other kind of differential equation you can imagine. 

A key issue to solving differential equations is *determining what kind of differential equation* one is trying to solve. Once you *know* a differential equation is in (or can be put in) **form $X$**, and if sympy has an algorithm to solve differential equations of **form $X$**, then sympy can quickly give the answer.  




In [ ]:
y=sp.Function('y')

sp.classify_ode( sp.Eq( y(x).diff(x) + ((3*x*y(x)+y(x)**2)/(x**2+x*y(x))), 0 ), y(x) )

The **classify_ode** routine returns the list of "types" of ODEs that the differential equation satisfies, which sympy knows how to solve.  If you find the default solution that sympy gives you is not satisfactory (perhaps the algebra is too complicated) you can ask sympy to solve your differential equation using another method, using the **hint** argument.